In [9]:
!pip install transformers -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.25.1-py3-none-any.whl (5.8 MB)
  Using cached huggingface_hub-0.11.1-py3-none-any.whl (182 kB)
  Using cached tokenizers-0.13.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.6 MB)


In [10]:
import pandas as pd
import re
from bs4 import BeautifulSoup

In [11]:
def clean_tweet(tweet):
    text=re.sub(r'@ [A-Za-z0-9\']+','',tweet)
    text=BeautifulSoup(text,'lxml').get_text()
    text=re.sub(r'https (//)[A-Za-z0-9. ]*(/) [A-Za-z0-9]+','',text)
    text=re.sub(r'https[A-Za-z0-9/. ]*','',text)
    text=re.sub("[^a-zA-Z]"," ",text)
    text=re.sub(r'\bRT\b',' ',text)
    text=re.sub(r'\bnan\b',' ',text)


    return text

In [12]:
#old_data=pd.read_csv('/content/texts.csv')

In [13]:
#old_data["Text"]=old_data["Text"].apply(clean_tweet)

In [14]:
#old_data

In [15]:
#text=old_data['Text'].tolist()
#label=old_data['class'].tolist()

In [16]:
#data = pd.DataFrame(list(zip(text, label)),
               #columns =['text', 'label'])
#data.head()

In [17]:
#data.to_csv("texts_required.csv",index=False)

In [18]:
#pd.read_csv('/content/texts_required.csv')

In [19]:
# https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/data
data = pd.read_csv("/content/merged_plus_texts.csv")
data

,text,label
0,pakistan ka ghra tauq he Pakistan Israel ko ta...,negative
1,Madarchod mulle ye mathura me Nahi dikha tha j...,negative
2,Manya Pradhan Mantri mahoday Shriman Narendra ...,positive
3,Krishna Jcb full trend me chal rahi aa,positive
4,Loksabha me janta sirf modi ko vote de rahi...,positive
...,...,...
44950,"Do not come out, this is Narendra Mudi Helle S...",negative
44951,Hariom Ho Electricity Ministers Pa Bills in Mi...,negative
44952,The name of a person with a strong muscle,negative
44953,"To behave badly, then why the barley of the we...",negative


In [20]:
y=list(data['label'])
for i in range(len(y)):
  if y[i]=="negative":
    y[i]=0
  elif y[i]=="neutral":
    y[i]=1
  else:
    y[i]=2

In [21]:
data['label']=y

In [22]:
data

,text,label
0,pakistan ka ghra tauq he Pakistan Israel ko ta...,0
1,Madarchod mulle ye mathura me Nahi dikha tha j...,0
2,Manya Pradhan Mantri mahoday Shriman Narendra ...,2
3,Krishna Jcb full trend me chal rahi aa,2
4,Loksabha me janta sirf modi ko vote de rahi...,2
...,...,...
44950,"Do not come out, this is Narendra Mudi Helle S...",0
44951,Hariom Ho Electricity Ministers Pa Bills in Mi...,0
44952,The name of a person with a strong muscle,0
44953,"To behave badly, then why the barley of the we...",0


In [23]:
data = data[['text','label']]
# data = data[0:1000]
data.head()

,text,label
0,pakistan ka ghra tauq he Pakistan Israel ko ta...,0
1,Madarchod mulle ye mathura me Nahi dikha tha j...,0
2,Manya Pradhan Mantri mahoday Shriman Narendra ...,2
3,Krishna Jcb full trend me chal rahi aa,2
4,Loksabha me janta sirf modi ko vote de rahi...,2


In [24]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [25]:
# from transformers import BertTokenizer, BertForSequenceClassification
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Seethal/sentiment_analysis_generic_dataset")

model = AutoModelForSequenceClassification.from_pretrained("Seethal/sentiment_analysis_generic_dataset")

Downloading:   0%|          | 0.00/333 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/711k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/769 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [26]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [27]:
model = model.to('cuda')

In [28]:
sample_data = ["I am eating","I am playing "]
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 1045, 2572, 5983, 102], [101, 1045, 2572, 2652, 102]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [29]:
X = list(data["text"])
y = list(data["label"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [30]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'attention_mask'])

In [31]:
print(X_train_tokenized['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [32]:
len(X_train),len(X_val)

(35964, 8991)

In [33]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["label"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [34]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [35]:
train_dataset[5]

{'input_ids': tensor([  101,  2748, 20368,  2378,  2003,  2088,  2452, 10556,  2707, 13433,
         28032,  7730,  1047,  7842,  8988,  1047, 12789, 27589,  2050,  2474,
          3654,  3527,  1062,  2953,  2099, 24532,  2050,  2079, 26822,  2099,
          2062, 13716, 16368,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [36]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred,average='micro')
    precision = precision_score(y_true=labels, y_pred=pred,average='micro')
    f1 = f1_score(y_true=labels, y_pred=pred,average='micro')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [37]:
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=6,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [38]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 35964
  Num Epochs = 6
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 26976
  Number of trainable parameters = 66955779


Step,Training Loss
500,1.004600
1000,0.945300
1500,0.912900
2000,0.886700
2500,0.896300
3000,0.876300
3500,0.897000
4000,0.889400
4500,0.871600
5000,0.734200


Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
Saving model checkpoint to output/checkpoint-1000
Configuration saved in output/checkpoint-1000/config.json
Model weights saved in output/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to output/checkpoint-1500
Configuration saved in output/checkpoint-1500/config.json
Model weights saved in output/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to output/checkpoint-2000
Configuration saved in output/checkpoint-2000/config.json
Model weights saved in output/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to output/checkpoint-2500
Configuration saved in output/checkpoint-2500/config.json
Model weights saved in output/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to output/checkpoint-3000
Configuration saved in output/checkpoint-3000/config.json
Model weights saved in output/check

TrainOutput(global_step=26976, training_loss=0.49304921290487014, metrics={'train_runtime': 1237.6201, 'train_samples_per_second': 174.354, 'train_steps_per_second': 21.797, 'total_flos': 9323575723511856.0, 'train_loss': 0.49304921290487014, 'epoch': 6.0})

In [39]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 8991
  Batch size = 8


<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 2.413825035095215,
 'eval_accuracy': 0.6166166166166166,
 'eval_precision': 0.6166166166166166,
 'eval_recall': 0.6166166166166166,
 'eval_f1': 0.6166166166166166,
 'eval_runtime': 8.4725,
 'eval_samples_per_second': 1061.204,
 'eval_steps_per_second': 132.665,
 'epoch': 6.0}

In [40]:
np.set_printoptions(suppress=True)

In [41]:
text = "That was good point"
# text = "go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[-3.1675, -0.0377,  2.4332]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.0034, 0.0777, 0.9189]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


array([[0.00339593, 0.07766156, 0.9189425 ]], dtype=float32)

In [50]:
trainer.save_model('/content/drive/MyDrive/tihsrha/CustomModel')

Saving model checkpoint to /content/drive/MyDrive/tihsrha/CustomModel
Configuration saved in /content/drive/MyDrive/tihsrha/CustomModel/config.json
Model weights saved in /content/drive/MyDrive/tihsrha/CustomModel/pytorch_model.bin


In [45]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/customModel

In [44]:
old_data=pd.read_csv('/content/timepass.csv')
old_data["Text"]=old_data["Text"].apply(clean_tweet)
test=old_data['Text'].tolist()
test
test_label=pd.read_csv('/content/Hinglish_test_labels.txt')
test_label

test_label=list(test_label['Sentiment'])
for i in range(len(test_label)):
  if test_label[i]=="negative":
    test_label[i]=0
  elif test_label[i]=="neutral":
    test_label[i]=1
  else:
    test_label[i]=2
test_label

[1,
 1,
 1,
 0,
 2,
 1,
 1,
 2,
 0,
 2,
 2,
 0,
 0,
 1,
 2,
 1,
 1,
 0,
 2,
 2,
 0,
 0,
 1,
 2,
 1,
 2,
 2,
 1,
 0,
 1,
 1,
 1,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 2,
 0,
 2,
 1,
 1,
 2,
 0,
 2,
 0,
 2,
 1,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 2,
 2,
 1,
 0,
 0,
 2,
 2,
 2,
 2,
 1,
 1,
 0,
 1,
 2,
 0,
 2,
 1,
 2,
 2,
 0,
 1,
 2,
 1,
 1,
 0,
 2,
 2,
 0,
 1,
 2,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 2,
 0,
 2,
 2,
 0,
 2,
 0,
 0,
 1,
 1,
 2,
 1,
 0,
 1,
 1,
 2,
 1,
 0,
 0,
 1,
 2,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 1,
 2,
 1,
 2,
 0,
 1,
 0,
 2,
 1,
 2,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 2,
 2,
 0,
 2,
 2,
 1,
 0,
 2,
 1,
 2,
 1,
 0,
 1,
 0,
 2,
 1,
 2,
 0,
 0,
 0,
 2,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 2,
 0,
 2,
 0,
 0,
 1,
 1,
 0,
 2,
 1,
 1,
 2,
 2,
 0,
 2,
 2,
 1,
 1,
 2,
 0,
 0,
 0,
 0,
 1,
 2,
 1,
 1,
 0,
 2,
 1,
 0,
 0,
 2,
 1,
 0,
 1,
 1,
 2,
 0,
 0,
 1,
 0,
 1,
 2,
 0,
 2,
 0,
 2,
 1,
 0,
 0,
 1,
 0,
 0,
 2,


In [49]:
model_2 = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/tihsrha/CustomModel")
model_2.to('cuda')

loading configuration file /content/drive/MyDrive/tihsrha/CustomModel/config.json
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Model config BertConfig {
  "_name_or_path": "Seethal/sentiment_analysis_generic_dataset",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "dim": 768,
  "dropout": 0.1,
  "hidden_act": "gelu",
  "hidden_dim": 3072,
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "n_heads": 12,
  "n_layers": 6,
  "num_attention_heads

ValueError: ignored

In [52]:
test_label

[1,
 1,
 1,
 0,
 2,
 1,
 1,
 2,
 0,
 2,
 2,
 0,
 0,
 1,
 2,
 1,
 1,
 0,
 2,
 2,
 0,
 0,
 1,
 2,
 1,
 2,
 2,
 1,
 0,
 1,
 1,
 1,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 2,
 0,
 2,
 1,
 1,
 2,
 0,
 2,
 0,
 2,
 1,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 2,
 2,
 1,
 0,
 0,
 2,
 2,
 2,
 2,
 1,
 1,
 0,
 1,
 2,
 0,
 2,
 1,
 2,
 2,
 0,
 1,
 2,
 1,
 1,
 0,
 2,
 2,
 0,
 1,
 2,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 2,
 0,
 2,
 2,
 0,
 2,
 0,
 0,
 1,
 1,
 2,
 1,
 0,
 1,
 1,
 2,
 1,
 0,
 0,
 1,
 2,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 1,
 2,
 1,
 2,
 0,
 1,
 0,
 2,
 1,
 2,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 2,
 2,
 0,
 2,
 2,
 1,
 0,
 2,
 1,
 2,
 1,
 0,
 1,
 0,
 2,
 1,
 2,
 0,
 0,
 0,
 2,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 2,
 0,
 2,
 0,
 0,
 1,
 1,
 0,
 2,
 1,
 1,
 2,
 2,
 0,
 2,
 2,
 1,
 1,
 2,
 0,
 0,
 0,
 0,
 1,
 2,
 1,
 1,
 0,
 2,
 1,
 0,
 0,
 2,
 1,
 0,
 1,
 1,
 2,
 0,
 0,
 1,
 0,
 1,
 2,
 0,
 2,
 0,
 2,
 1,
 0,
 0,
 1,
 0,
 0,
 2,


In [51]:
test

['  Agr kse ko itni importantce chaeay ni tou     ',
 'logon ko alloo pyaz tomator me toh allah pak ka naam nazar aa jata hai pr aankhon k samne allah pak ke bande nazar     ',
 'Wafadaar bane rahane ka nayab tarika hai lamba ji aap us party se Judi hai jiska sardar      ',
 'Chup bhosdike   He has been exonerated of all the charges   Tum log ghanta prove nahi kar paaye  ',
 '                                                                                            Praise the Great Giver who gives sustenance to all   Paid obeisance at Gurudwara Sr    ',
 '  HNY  ki announcement hogyi to humka iss gola me nhi rehna hum nikal jaounga dusre gola me',
 '   We re almost in the final three days of Ramadan     very close to the finish line   So let s make sure that we double up our I    ',
 'This song makes me super happy   I sing it for myself   Take   Khush raho abaad raho   Hemant Da blessings   ',
 'tm dikha rhy ho apni asli auqaat wese btao tou ye propaganda kyu krta hai shahid masood  

In [55]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score,f1_score

In [63]:
# text = "That was good point"
preds = []
for txt in tqdm(test):
#text = "pakistan ka ghra tauq he Pakistan Israel ko"
  inputs = tokenizer(txt,padding = True, truncation = True, return_tensors='pt').to('cuda')
  outputs = model(**inputs)
  predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
  predictions = predictions.cpu().detach().numpy()
  predictions = np.argmax(predictions,axis=1)
  preds.append(int(predictions))
  #print(predictions.shape)
  #preds = np.concatenate(preds,np.array(predictions))
  

#predictions

100%|██████████| 3000/3000 [00:26<00:00, 112.64it/s]


In [64]:
preds

[1,
 1,
 2,
 0,
 2,
 0,
 1,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 0,
 1,
 0,
 1,
 0,
 2,
 0,
 1,
 2,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 2,
 2,
 1,
 0,
 0,
 2,
 2,
 0,
 2,
 1,
 2,
 2,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 0,
 1,
 1,
 2,
 1,
 0,
 2,
 1,
 2,
 0,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 0,
 1,
 0,
 0,
 2,
 1,
 1,
 1,
 0,
 1,
 2,
 0,
 1,
 1,
 2,
 0,
 1,
 0,
 1,
 1,
 2,
 1,
 2,
 1,
 0,
 2,
 1,
 0,
 1,
 1,
 2,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 2,
 2,
 1,
 2,
 1,
 0,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 0,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 1,
 0,
 1,
 0,
 0,
 0,
 2,
 1,
 0,
 2,
 0,
 1,
 0,
 0,
 2,
 2,
 0,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 1,
 2,
 0,
 2,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 2,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 2,
 1,
 1,
 1,
 0,
 0,
 2,
 2,
 0,
 1,
 2,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 2,
 1,
 2,
 1,
 0,
 1,
 2,
 0,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 0,
 2,
 0,
 2,
 1,
 2,
 0,
 1,
 2,
 0,
 2,


In [65]:
score = accuracy_score(test_label,preds)

In [66]:
score

0.6393333333333333

In [68]:
score = f1_score(test_label,preds,average='micro')

In [69]:
score

0.6393333333333333